In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e16/train.csv')
train_df.head()

### Sex
M:オス,F:メス,I:未成熟
### Length
前後の長さ
### Diameter
身体の厚み
### Height
高さ
### Weight
重さ
### Shucked Weight
可食部の重さ
### Viscera Weight
内臓器官の重さ
### Shell Weight
外殻の重さ
### Age
年齢、**今回のターゲット**

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
sex_mapping = {
    'M':0, 
    'F':1,
    'I':2,
}

In [ ]:
train_df['Sex'] = train_df['Sex'].map(sex_mapping)
train_df['Sex'].value_counts()

In [ ]:
train_df.dtypes

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

target = 'Age'
train_y = train_data[target]
train_x = train_data.drop(target, axis=1)

val_y = val_data[target]
val_x = val_data.drop(target, axis=1)

train = lgb.Dataset(train_x, train_y)
val = lgb.Dataset(val_x, val_y)

params = {
    'objective':'regression',
    'metrics':'mae',
    'boosting_type':'gbdt',
    'learning_rate':0.01
}

model = lgb.train(params, train, valid_sets=val, num_boost_round=1000, early_stopping_rounds=100)

In [ ]:
pred_y = model.predict(val_x)
pred_y

In [ ]:
mae(val_y, pred_y)

In [ ]:
test_data = pd.read_csv('/kaggle/input/playground-series-s3e16/test.csv')
test_data['Sex'] = test_data['Sex'].map(sex_mapping)


In [ ]:
predict = model.predict(test_data)
test_data['Age'] = predict

In [ ]:
filter_col = ['id', 'Age']
df = test_data[filter_col]

In [ ]:
df.to_csv('sample_submission.csv', index=False)